In [2]:
import autorootcwd

In [3]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from src.data import heart_disease, titanic, artificial

In [4]:
# prepare the data
X_heart, y_heart = heart_disease.prepare_data()
X_titanic, y_titanic = titanic.prepare_data()
X_artificial, y_artificial, true_weights = artificial.generate_data(
    num_samples=2000, num_features=5, interaction_pairs=[(1, 3), (2, 4)]
)

# split the data into train and test sets
X_train_heart, X_test_heart, y_train_heart, y_test_heart = train_test_split(
    X_heart,
    y_heart,
    test_size=0.2,
    random_state=42,
)
X_train_titanic, X_test_titanic, y_train_titanic, y_test_titanic = train_test_split(
    X_titanic,
    y_titanic,
    test_size=0.2,
    random_state=42,
)
X_train_artificial, X_test_artificial, y_train_artificial, y_test_artificial = train_test_split(
    X_artificial,
    y_artificial,
    test_size=0.2,
    random_state=42,
)

In [ ]:
# Create classifiers
lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()
knn = KNeighborsClassifier()

# Train classifiers
lda.fit(X_train, y_train)
qda.fit(X_train, y_train)
knn.fit(X_train, y_train)

# Make predictions
lda_preds = lda.predict(X_test)
qda_preds = qda.predict(X_test)
knn_preds = knn.predict(X_test)

In [ ]:
# train and evaluate the logistic regression model
log_reg = LogisticRegression()
log_reg.fit(X_train_heart, y_train_heart)
y_pred_heart = log_reg.predict(X_test_heart)
acc_log_reg_heart = accuracy_score(y_test_heart, y_pred_heart)
f1_log_reg_heart = f1_score(y_test_heart, y_pred_heart)
roc_auc_log_reg_heart = roc_auc_score(y_test_heart, y_pred_heart)

log_reg.fit(X_train_titanic, y_train_titanic)
y_pred_titanic = log_reg.predict(X_test_titanic)
acc_log_reg_titanic = accuracy_score(y_test_titanic, y_pred_titanic)
f1_log_reg_titanic = f1_score(y_test_titanic, y_pred_titanic)
roc_auc_log_reg_titanic = roc_auc_score(y_test_titanic, y_pred_titanic)

log_reg.fit(X_train_artificial, y_train_artificial)
y_pred_artificial = log_reg.predict(X_test_artificial)
acc_log_reg_artificial = accuracy_score(y_test_artificial, y_pred_artificial)
f1_log_reg_artificial = f1_score(y_test_artificial, y_pred_artificial)
roc_auc_log_reg_artificial = roc_auc_score(y_test_artificial, y_pred_artificial)

# train and evaluate the LDA model
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_heart, y_train_heart)
y_pred_heart = lda.predict(X_test_heart)
acc_lda_heart = accuracy_score(y_test_heart, y_pred_heart)
f1_lda_heart = f1_score(y_test_heart, y_pred_heart)
roc_auc_lda_heart = roc_auc_score(y_test_heart, y_pred_heart)

lda.fit(X_train_titanic, y_train_titanic)
y_pred_titanic = lda.predict(X_test_titanic)
acc_lda_titanic = accuracy_score(y_test_titanic, y_pred_titanic)
f1_lda_titanic = f1_score(y_test_titanic, y_pred_titanic)
roc_auc_lda_titanic = roc_auc_score(y_test_titanic, y_pred_titanic)
